In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dlt
import os
import random
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.datasets import cifar10
from keras.utils import np_utils

from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.core import Dense,Activation,Dropout,Flatten
from keras.datasets import cifar10
from keras.utils import np_utils


2022-11-28 16:30:23.453888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 16:30:23.529052: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-28 16:30:23.548151: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-28 16:30:23.870500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
#データセットを読み込んでテンソルに変換

#画像名リスト作成
#ここで指定したディレクトリをまとめてnp配列に変換できる。

#dcgan_filename = os.listdir('./10000all')
dcgan_filename = os.listdir('./20000dcpic')

true_filename = os.listdir('./Ex_4')

test_filename = os.listdir('./500test')

#クラスリスト作成
#ex:"frog"が入ってたら6
def makeclass(filename):
    list1 = []
    
    for name in filename:
        if "airplane" in name:
            list1.append([0])
        elif "automobile" in name:
            list1.append([1])
        elif "bird" in name:
            list1.append([2])
        elif "cat" in name:
            list1.append([3])
        elif "deer" in name:
            list1.append([4])
        elif "dog" in name:
            list1.append([5])
        elif "frog" in name:
            list1.append([6])
        elif "horse" in name:
            list1.append([7])
        elif "ship" in name:
            list1.append([8])
        elif "truck" in name:
            list1.append([9])
        else:
            list1.append([10])

            
    #list型をnp.arrayに変換
    classlist = np.array(list1)
    return classlist

if ".ipynb_checkpoints" in dcgan_filename:
    dcgan_filename.remove(".ipynb_checkpoints")     #いらないものを消す。

if ".ipynb_checkpoints" in true_filename:
    true_filename.remove(".ipynb_checkpoints")

if ".ipynb_checkpoints" in test_filename:
    test_filename.remove(".ipynb_checkpoints")
#print(filename)


#統計用リスト
lskekka = []
ackekka = []

for run in range(10):

    dcgan_label = makeclass(dcgan_filename)
    true_label = makeclass(true_filename)
    test_label = makeclass(test_filename)


    #画像ファイルの相対パスを取得
    dcgan_filepass = []
    for name in dcgan_filename:
        #dcgan_filepass.append("./10000all/"+name)
        dcgan_filepass.append("./20000dcpic/"+name)


    #print(filepass)

    true_filepass = []
    for name in true_filename:
        true_filepass.append("./Ex_4/"+name)


    test_filepass = []
    for name in test_filename:
        test_filepass.append("./500test/"+name)


    #dcgan画像
    #png画像をndarrayに変換。→一旦listに直して、imlistに追加
    dcgan_imlist = []
    for im in dcgan_filepass:
        pic = np.array(Image.open(im))
        piclist = pic.tolist()
        dcgan_imlist.append(piclist)

    #imlist（list型）をimarray（ndarray型）に変換
    dcgan_imarray = np.array(dcgan_imlist)


    #true画像
    #png画像をndarrayに変換。→一旦listに直して、imlistに追加
    true_imlist = []
    for im in true_filepass:
        pic = np.array(Image.open(im))
        piclist = pic.tolist()
        true_imlist.append(piclist)

    #imlist（list型）をimarray（ndarray型）に変換
    true_imarray = np.array(true_imlist)

    #print(np.concatenate([dcgan_imarray, true_imarray]))

    #test画像
    test_imlist = []

    for im in test_filepass:
        pic = np.array(Image.open(im))
        piclist = pic.tolist()
        test_imlist.append(piclist)

    #imlist（list型）をimarray（ndarray型）に変換
    test_imarray = np.array(test_imlist)

    #使用画像選択!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #Ex_2:CIFAR−１０画像+TP画像2500枚+FP画像2500枚のパターン。
    #
    
    
    xttr = true_imarray
    yttr = true_label

    xtcon = np.concatenate([true_imarray[0:5000],dcgan_imarray[0:5000]])
    ytcon = np.concatenate([true_label[0:5000], dcgan_label[0:5000]])

    (x_train, y_train) = (xttr, yttr)
    (x_test, y_test) = (test_imarray, test_label)


    #画像を0-1の範囲で正規化
    x_train=x_train.astype('float32')/255.0
    x_test=x_test.astype('float32')/255.0

    #正解ラベルをOne-Hot表現に変換
    y_train=np_utils.to_categorical(y_train,10)
    y_test=np_utils.to_categorical(y_test,10)

    #モデルを構築
    model=Sequential()

    model.add(Conv2D(64,(3,3),padding='same',input_shape=(32,32,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(128,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    #model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,activation='softmax'))

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

    history=model.fit(x_train,y_train,batch_size=128,epochs=20,verbose=1,validation_split=0.1)

    #モデルと重みを保存
    json_string=model.to_json()
    open('cifar10_cnn.json',"w").write(json_string)
    model.save_weights('cifar10_cnn.h5')

    #モデルの表示
    #model.summary()

    #評価
    score=model.evaluate(x_test,y_test,verbose=0)
    print('Test loss:',score[0])
    print('Test accuracy:',score[1])
    
    lskekka.append(score[0])
    ackekka.append(score[1])
    



2022-11-28 16:30:45.193020: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-28 16:30:45.211207: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-28 16:30:45.211301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-28 16:30:45.211611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Epoch 1/20


2022-11-28 16:30:46.533821: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2022-11-28 16:30:46.988220: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


71/71 [==============================] - 3s 16ms/step - loss: 1.8808 - accuracy: 0.3074 - val_loss: 1.6250 - val_accuracy: 0.4250
Epoch 2/20
71/71 [==============================] - 1s 11ms/step - loss: 1.4729 - accuracy: 0.4614 - val_loss: 1.4344 - val_accuracy: 0.4770
Epoch 3/20
71/71 [==============================] - 1s 11ms/step - loss: 1.2664 - accuracy: 0.5486 - val_loss: 1.3095 - val_accuracy: 0.5120
Epoch 4/20
71/71 [==============================] - 1s 11ms/step - loss: 1.0828 - accuracy: 0.6130 - val_loss: 1.0867 - val_accuracy: 0.6190
Epoch 5/20
71/71 [==============================] - 1s 11ms/step - loss: 0.9077 - accuracy: 0.6830 - val_loss: 1.0620 - val_accuracy: 0.6480
Epoch 6/20
71/71 [==============================] - 1s 11ms/step - loss: 0.7286 - accuracy: 0.7426 - val_loss: 1.0110 - val_accuracy: 0.6510
Epoch 7/20
71/71 [==============================] - 1s 11ms/step - loss: 0.5715 - accuracy: 0.7994 - val_loss: 0.9955 - val_accuracy: 0.6700
Epoch 8/20
71/71 [======

In [3]:
print(lskekka)
print(ackekka)

print(np.average(ackekka))

[2.7281532287597656, 2.812262535095215, 2.8997950553894043, 2.666116714477539, 2.7751870155334473, 2.97129225730896, 2.9893364906311035, 2.9269754886627197, 2.869988441467285, 2.802673816680908]
[0.5799999833106995, 0.5699999928474426, 0.5879999995231628, 0.5580000281333923, 0.5830000042915344, 0.5600000023841858, 0.5699999928474426, 0.5809999704360962, 0.5659999847412109, 0.5529999732971191]
0.5708999931812286


In [4]:
import pandas as pd
def counter(x):
    count = [0,0,0,0,0,0,0,0,0,0]
    for i in range(len(x)):
        if x[i] == 0:
            count[0] += 1
        elif x[i] == 1:
            count[1] += 1
        elif x[i] == 2:
            count[2] += 1
        elif x[i] == 3:
            count[3] += 1
        elif x[i] == 4:
            count[4] += 1
        elif x[i] == 5:
            count[5] += 1
        elif x[i] == 6:
            count[6] += 1
        elif x[i] == 7:
            count[7] += 1
        elif x[i] == 8:
            count[8] += 1
        else:
            count[9] += 1
    s1 = pd.Series(count)
    return s1
    
a = np_utils.to_categorical(dcgan_label[0:5000],10)
print(counter(np.argmax(a,1)))

0    489
1    484
2    500
3    474
4    505
5    488
6    494
7    521
8    537
9    508
dtype: int64
